In [ ]:
%load_ext autoreload
%autoreload 2

import os
import nibabel as nib
import numpy as np
from skimage.io import imread
from skimage.io import imshow
import matplotlib.pyplot as plt

In [ ]:
from skimage.transform import resize
from sklearn.preprocessing import normalize

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import torch as tc
# tc.__version__

In [ ]:
from models import Mednet

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
# Load image
# Citation : Case courtesy of A.Prof Frank Gaillard, Radiopaedia.org, rID: 33753
path = "~/Documents/SCU/workshops/AIMI/alzheimer-disease-ct-only.png"
img = imread(path)

In [ ]:
# Display image
imshow(img)

In [ ]:
# Looks like the original image has 4 channels
img.shape

In [ ]:
# I am going to arbitrarily select one channel
# Note : Do not select the last channel
img_ch = img[:,:,0]
# 1 channel image
img_ch.shape

In [ ]:
# Sanity check
imshow(img_ch)

In [ ]:
# Normalize image
# img_ch = normalize(img_ch)

In [ ]:
# I resized to (96, 96) because I trained the network on (96, 96) images
shape = (96, 96)
img_ch = resize(img_ch, shape)

In [ ]:
# Sanity Check
# Well, we squished the brain, but it is still recognizable as a brain
# Would it be better to crop the image? Perhaps, select a (96, 96) portion of the image?
plt.imshow(img_ch)

In [ ]:
img_ch = tc.Tensor(img_ch).float()

In [ ]:
img_ch = tc.unsqueeze(img_ch, dim = 0)

In [ ]:
img_ch = tc.unsqueeze(img_ch, dim = 0)

In [ ]:
# Sanity Check
img_ch.shape

In [ ]:
# Now, we are ready to pass the image into the network!

In [ ]:
####
# Load the network

In [ ]:
# Load model
model = Mednet()

In [ ]:
# Let's look at the model that we are using.
# repr(model)

In [ ]:
# Load model weights
path = "pretrained_weights.pth"
model.load_state_dict(tc.load(path))

In [ ]:
####
# Unfortunately, I cannot redistribute the dataset to train and test the network.
####

# TPR = Sensitivity
# Sensitivity = Recall = TP/(TP + FN) = (# guessed positive and correct)/(# actually positive) = How many positive cases you caught
# If Recall is low, FN is high and a lot of positive cases slipped through the cracks

# FPR = 1 - Specificity
# Specificity TN/(TN + FP) = (# guessed negative and correctly)/(# actually negative) = How many negative cases you caught
# FPR = FP/(TN + FP) = (# guess positive and incorrectly)/(# actually negative)
# FPR = 1 - How many negative cases you caught = How many negative cases slipped through the cracks

# TP : Guessed P and were right
# TN : Guessed N and were right
# FP : Guessed P and were wrong
# FN : Guessed N and were wrong

# ROC Curve
# TPR = How many positive case you caught
# FPR = How many negative cases fell through the cracks
# Ideally, TPR = 1, meaning you caught all positive cases and have no FN!!!
# & FPR = 0, meaning none of the negative cases slipped through the cracks!!!

In [ ]:
# Class Activation Mapping (CAM)

In [ ]:
# Extract layer names and weights
keys = list(model.state_dict().keys())
values = list(model.state_dict().values())

In [ ]:
# print(keys[len(keys) - 1])
print(keys[-1], values[-1])
# Class 0 weight
# Class 1 weight

In [ ]:
w0 = values[-1][0]
w1 = values[-1][1]

In [ ]:
print(w0, w1)

In [ ]:
# Extract feature maps from last convolutional layer
probs, features = model(img_ch)

In [ ]:
features.shape

In [ ]:
# Results in a CAM for each class
# Each CAM allows us to see what features contribute to the class prediction
CAM0 = tc.zeros(features.shape[2], features.shape[3])
CAM1 = tc.zeros(features.shape[2], features.shape[3])

idx = 0 # a specific sample in the batch
for i in range(features.shape[1]):
    CAM0 += features[idx][i]*w0
    CAM1 += features[idx][i]*w1

In [ ]:
# Resize Class Activation Maps if the network downsamples the image

In [ ]:
plt.imshow(CAM0.detach().numpy())

In [ ]:
plt.imshow(CAM1.detach().numpy())